<a href="https://colab.research.google.com/github/yohannesb/AdvSR/blob/master/saturday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install update pip
!pip uninstall -y kapre keras tensorflow-probability albumentations datascience h5py keras-nightly # Avoid crashes with pre-installed packages 
!git clone https://github.com/lvapeab/nmt-keras
import os
os.chdir('nmt-keras')
!pip install -e .


Found existing installation: kapre 0.3.6
Uninstalling kapre-0.3.6:
  Successfully uninstalled kapre-0.3.6
Found existing installation: keras 2.7.0
Uninstalling keras-2.7.0:
  Successfully uninstalled keras-2.7.0
Found existing installation: tensorflow-probability 0.15.0
Uninstalling tensorflow-probability-0.15.0:
  Successfully uninstalled tensorflow-probability-0.15.0
Found existing installation: albumentations 0.1.12
Uninstalling albumentations-0.1.12:
  Successfully uninstalled albumentations-0.1.12
Found existing installation: datascience 0.10.6
Uninstalling datascience-0.10.6:
  Successfully uninstalled datascience-0.10.6
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Successfully uninstalled h5py-3.1.0
Cloning into 'nmt-keras'...
remote: Enumerating objects: 4807, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 4807 (delta 31), reused 30 (delta 15), pack-reused 4730
Receiving objects: 100% (480

In [1]:
from keras_wrapper.dataset import Dataset, saveDataset
from data_engine.prepare_data import keep_n_captions
ds = Dataset('tutorial_dataset', 'tutorial', silence=False)

Using TensorFlow backend.


In [4]:
pwd

'/content'

In [5]:
ls

nmt-keras/  sample_data/


In [6]:
cd nmt-keras/ 

/content/nmt-keras


In [7]:
ls

config.py     examples/    meta-optimizers/     sample_ensemble.py  utils/
data_engine/  __init__.py  nmt_keras/           score.py
demo-web/     LICENSE      nmt_keras.egg-info/  setup.py
docs/         main.py      README.md            tests/


In [8]:
cd examples/  

/content/nmt-keras/examples


In [9]:
ls

configs/        EuTrans/                 README.md
documentation/  modeling_tutorial.ipynb  tutorial.ipynb


In [10]:
cd ..

/content/nmt-keras


In [11]:
ds.setOutput('examples/EuTrans/training.en',
             'train',
             type='text',
             id='target_text',
             tokenization='tokenize_none',
             build_vocabulary=True,
             pad_on_batch=True,
             sample_weights=True,
             max_text_len=30,
             max_words=30000,
             min_occ=0)


[18/12/2021 07:15:07] 	Applying tokenization function: "tokenize_none".
[18/12/2021 07:15:07] Creating vocabulary for data with data_id 'target_text'.
[18/12/2021 07:15:07] 	 Total: 513 unique words in 9900 sentences with a total of 98304 words.
[18/12/2021 07:15:07] Creating dictionary of 30000 most common words, covering 100.0% of the text.
[18/12/2021 07:15:07] Loaded "train" set outputs of data_type "text" with data_id "target_text" and length 9900.


In [12]:
ds.setOutput('examples/EuTrans/dev.en',
             'val',
             type='text',
             id='target_text',
             pad_on_batch=True,
             tokenization='tokenize_none',
             sample_weights=True,
             max_text_len=30,
             max_words=0)

[18/12/2021 07:15:27] 	Applying tokenization function: "tokenize_none".
[18/12/2021 07:15:27] Loaded "val" set outputs of data_type "text" with data_id "target_text" and length 100.


In [13]:
ds.setInput('examples/EuTrans/training.es',
            'train',
            type='text',
            id='source_text',
            pad_on_batch=True,
            tokenization='tokenize_none',
            build_vocabulary=True,
            fill='end',
            max_text_len=30,
            max_words=30000,
            min_occ=0)
ds.setInput('examples/EuTrans/dev.es',
            'val',
            type='text',
            id='source_text',
            pad_on_batch=True,
            tokenization='tokenize_none',
            fill='end',
            max_text_len=30,
            min_occ=0)

[18/12/2021 07:16:24] 	Applying tokenization function: "tokenize_none".
[18/12/2021 07:16:24] Creating vocabulary for data with data_id 'source_text'.
[18/12/2021 07:16:24] 	 Total: 686 unique words in 9900 sentences with a total of 96172 words.
[18/12/2021 07:16:24] Creating dictionary of 30000 most common words, covering 100.0% of the text.
[18/12/2021 07:16:24] Loaded "train" set inputs of data_type "text" with data_id "source_text" and length 9900.
[18/12/2021 07:16:24] 	Applying tokenization function: "tokenize_none".
[18/12/2021 07:16:24] Loaded "val" set inputs of data_type "text" with data_id "source_text" and length 100.


In [14]:
ds.setInput('examples/EuTrans/training.en',
            'train',
            type='text',
            id='state_below',
            required=False,
            tokenization='tokenize_none',
            pad_on_batch=True,
            build_vocabulary='target_text',
            offset=1,
            fill='end',
            max_text_len=30,
            max_words=30000)
ds.setInput(None,
            'val',
            type='ghost',
            id='state_below',
            required=False)

[18/12/2021 07:16:47] 	Applying tokenization function: "tokenize_none".
[18/12/2021 07:16:47] 	Reusing vocabulary named "target_text" for data with data_id "state_below".
[18/12/2021 07:16:47] Loaded "train" set inputs of data_type "text" with data_id "state_below" and length 9900.
[18/12/2021 07:16:47] Loaded "val" set inputs of data_type "ghost" with data_id "state_below" and length 100.


In [15]:
for split, input_text_filename in zip(['train', 'val'], ['examples/EuTrans/training.es', 'examples/EuTrans/dev.es']):
    ds.setRawInput(input_text_filename,
                  split,
                  type='file-name',
                  id='raw_source_text',
                  overwrite_split=True)

[18/12/2021 07:17:08] Loaded "train" set inputs of type "file-name" with id "raw_source_text".
[18/12/2021 07:17:08] Loaded "val" set inputs of type "file-name" with id "raw_source_text".


In [16]:
keep_n_captions(ds, repeat=1, n=1, set_names=['val'])

[18/12/2021 07:17:30] Keeping 1 captions per input on the val set.
[18/12/2021 07:17:30] Samples reduced to 100 in val set.


In [17]:
saveDataset(ds, 'datasets')

[18/12/2021 07:17:52] <<< creating directory datasets ... >>>
[18/12/2021 07:17:52] <<< Saving Dataset instance to datasets/Dataset_tutorial_dataset.pkl ... >>>
[18/12/2021 07:17:52] <<< Dataset instance saved >>>


In [18]:
from config import load_parameters
from nmt_keras.model_zoo import TranslationModel
from keras_wrapper.cnn_model import loadModel
from keras_wrapper.dataset import loadDataset
from keras_wrapper.extra.callbacks import PrintPerformanceMetricOnEpochEndOrEachNUpdates
params = load_parameters()
dataset = loadDataset('datasets/Dataset_tutorial_dataset.pkl')

[18/12/2021 07:18:24] <<< Cupy not available. Using numpy. >>>
[18/12/2021 07:18:24] <<< Loading Dataset instance from datasets/Dataset_tutorial_dataset.pkl ... >>>
[18/12/2021 07:18:24] <<< Dataset instance loaded >>>


In [19]:
params['INPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['source_text']
params['OUTPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len['target_text']

In [20]:
params['MODEL_TYPE'] = 'AttentionRNNEncoderDecoder' #  Supported models: 'AttentionRNNEncoderDecoder' and 'Transformer'.
nmt_model = TranslationModel(params,
                             model_type=params['MODEL_TYPE'], 
                             model_name='tutorial_model',
                             vocabularies=dataset.vocabulary,
                             store_path='trained_models/tutorial_model/',
                             verbose=True)

[18/12/2021 07:19:05] <<< Building AttentionRNNEncoderDecoder Translation_Model >>>


[18/12/2021 07:19:05] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:650: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



[18/12/2021 07:19:05] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:4786: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



[18/12/2021 07:19:05] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:157: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[18/12/2021 07:19:06] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:3561: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


-----------------------------------------------------------------------------------
		TranslationModel instance
-----------------------------------------------------------------------------------
_model_type: AttentionRNNEncoderDecoder
name: tutorial_model
model_path: trained_models/tutorial_model/
verbose: True

Params:
	ACCUMULATE_GRADIENTS: 1
	ADDITIONAL_OUTPUT_MERGE_MODE: Add
	ALIGN_FROM_RAW: True
	ALPHA_FACTOR: 0.6
	AMSGRAD: False
	APPLY_DETOKENIZATION: False
	ATTENTION_DROPOUT_P: 0.0
	ATTENTION_MODE: add
	ATTENTION_SIZE: 32
	BATCH_NORMALIZATION_MODE: 1
	BATCH_SIZE: 50
	BEAM_SEARCH: True
	BEAM_SIZE: 6
	BETA_1: 0.9
	BETA_2: 0.999
	BIDIRECTIONAL_DEEP_ENCODER: True
	BIDIRECTIONAL_ENCODER: True
	BIDIRECTIONAL_MERGE_MODE: concat
	BPE_CODES_PATH: examples/EuTrans//training_codes.joint
	CLASSIFIER_ACTIVATION: softmax
	CLIP_C: 5.0
	CLIP_V: 0.0
	COVERAGE_NORM_FACTOR: 0.2
	COVERAGE_PENALTY: False
	DATASET_NAME: EuTrans
	DATASET_STORE_PATH: datasets/
	DATA_AUGMENTATION: False
	DATA_ROOT_PATH

[18/12/2021 07:19:06] From /content/nmt-keras/nmt_keras/model_zoo.py:213: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

[18/12/2021 07:19:07] Preparing optimizer and compiling. Optimizer configuration: 
	 LR: 0.001
	 LOSS: categorical_crossentropy
	 BETA_1: 0.9
	 BETA_2: 0.999
	 EPSILON: 1e-08


[18/12/2021 07:19:07] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:1192: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



In [21]:
inputMapping = dict()
for i, id_in in enumerate(params['INPUTS_IDS_DATASET']):
    pos_source = dataset.ids_inputs.index(id_in)
    id_dest = nmt_model.ids_inputs[i]
    inputMapping[id_dest] = pos_source
nmt_model.setInputsMapping(inputMapping)

outputMapping = dict()
for i, id_out in enumerate(params['OUTPUTS_IDS_DATASET']):
    pos_target = dataset.ids_outputs.index(id_out)
    id_dest = nmt_model.ids_outputs[i]
    outputMapping[id_dest] = pos_target
nmt_model.setOutputsMapping(outputMapping)


In [22]:
is_transformer = params.get('ATTEND_ON_OUTPUT', 'transformer' in params['MODEL_TYPE'].lower())
search_params = {
    'language': 'en',
    'tokenize_f': eval('dataset.' + 'tokenize_none'),
    'beam_size': 12,
    'optimized_search': True,
    'model_inputs': params['INPUTS_IDS_MODEL'],
    'model_outputs': params['OUTPUTS_IDS_MODEL'],
    'dataset_inputs':  params['INPUTS_IDS_DATASET'],
    'dataset_outputs':  params['OUTPUTS_IDS_DATASET'],
    'n_parallel_loaders': 1,
    'maxlen': 50,
    'normalize_probs': True,
    'pos_unk': True and not is_transformer,  # Pos_unk is unimplemented for transformer models
    'heuristic': 0,
    'state_below_maxlen': -1,
    'attend_on_output': is_transformer,
    'val': {'references': dataset.extra_variables['val']['target_text']}
  }

vocab = dataset.vocabulary['target_text']['idx2words']
callbacks = []
input_text_id = params['INPUTS_IDS_DATASET'][0]

callbacks.append(PrintPerformanceMetricOnEpochEndOrEachNUpdates(nmt_model,
                                                                dataset,
                                                                gt_id='target_text',
                                                                metric_name=['sacrebleu'],
                                                                set_name=['val'],
                                                                batch_size=50,
                                                                each_n_epochs=1,
                                                                extra_vars=search_params,
                                                                reload_epoch=0,
                                                                is_text=True,
                                                                input_text_id=input_text_id,
                                                                index2word_y=vocab,
                                                                sampling_type='max_likelihood',
                                                                beam_search=True,
                                                                save_path=nmt_model.model_path,
                                                                start_eval_on_epoch=0,
                                                                write_samples=True,
                                                                write_type='list',
                                                                verbose=True))

In [23]:
training_params = {'n_epochs': 4,
                   'batch_size': 50,
                   'maxlen': 30,
                   'epochs_for_save': 1,
                   'verbose': 1,
                   'eval_on_sets': [], 
                   'n_parallel_loaders': 1,
                   'extra_callbacks': callbacks,
                   'reload_epoch': 0,
                   'epoch_offset': 0}

In [24]:
nmt_model.trainNet(dataset, training_params)

[18/12/2021 07:20:55] <<< Training model >>>
[18/12/2021 07:20:55] Training parameters: { 
	batch_size: 50
	class_weights: None
	da_enhance_list: []
	da_patch_type: resize_and_rndcrop
	data_augmentation: False
	each_n_epochs: 1
	epoch_offset: 0
	epochs_for_save: 1
	eval_on_epochs: True
	eval_on_sets: []
	extra_callbacks: [<keras_wrapper.extra.callbacks.EvalPerformance object at 0x7f9181f30ed0>]
	homogeneous_batches: False
	initial_lr: 1.0
	joint_batches: 4
	lr_decay: None
	lr_gamma: 0.1
	lr_half_life: 50000
	lr_reducer_exp_base: 0.5
	lr_reducer_type: linear
	lr_warmup_exp: -1.5
	maxlen: 30
	mean_substraction: False
	metric_check: None
	min_delta: 0.0
	min_lr: 1e-09
	n_epochs: 4
	n_gpus: 1
	n_parallel_loaders: 1
	normalization_type: None
	normalize: False
	num_iterations_val: None
	patience: 0
	patience_check_split: val
	reduce_each_epochs: True
	reload_epoch: 0
	shuffle: True
	start_eval_on_epoch: 0
	start_reduction_on_epoch: 0
	tensorboard: False
	tensorboard_params: {'log_dir': 'tens

[18/12/2021 07:20:59] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:3315: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



[18/12/2021 07:20:59] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:292: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



[18/12/2021 07:20:59] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:299: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



[18/12/2021 07:20:59] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:312: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



...

[18/12/2021 07:22:52] <<< Saving model to trained_models/tutorial_model/epoch_4 ... >>>


[18/12/2021 07:22:52] <<< Model saved >>>

[18/12/2021 07:22:52] <<< Predicting outputs of val set >>>




 Total cost: 239.296089 	 Average cost: 2.392961
The sampling took: 4.370353 secs (Speed: 0.043704 sec/sample)


[18/12/2021 07:22:57] Prediction output 0: target_text (text)
[18/12/2021 07:22:57] Decoding beam search prediction ...
[18/12/2021 07:22:57] Using heuristic 0
[18/12/2021 07:22:57] Evaluating on metric sacrebleu
[18/12/2021 07:22:57] Computing SacreBleu scores on the val split...
[18/12/2021 07:22:57] Bleu_4: 73.87678870339836


[18/12/2021 07:22:57] Done evaluating on metric sacrebleu
[18/12/2021 07:22:57] 
<<< Progress plot saved in trained_models/tutorial_model/epoch_4.jpg >>>
[18/12/2021 07:22:57] <<< Finished training model >>>


In [25]:
dataset.setInput('examples/EuTrans/test.es',
            'test',
            type='text',
            id='source_text',
            pad_on_batch=True,
            tokenization='tokenize_none',
            fill='end',
            max_text_len=30,
            min_occ=0)

dataset.setInput(None,
            'test',
            type='ghost',
            id='state_below',
            required=False)

dataset.setRawInput('examples/EuTrans/test.es',
              'test',
              type='file-name',
              id='raw_source_text',
              overwrite_split=True)


[18/12/2021 07:35:39] 	Applying tokenization function: "tokenize_none".
[18/12/2021 07:35:39] Loaded "test" set inputs of data_type "text" with data_id "source_text" and length 2996.
[18/12/2021 07:35:39] Loaded "test" set inputs of data_type "ghost" with data_id "state_below" and length 2996.
[18/12/2021 07:35:39] Loaded "test" set inputs of type "file-name" with id "raw_source_text".


In [26]:
params['INPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len[params['INPUTS_IDS_DATASET'][0]]
params['OUTPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len[params['OUTPUTS_IDS_DATASET'][0]]

In [27]:
# Load model
nmt_model = loadModel('trained_models/tutorial_model', 4)

[18/12/2021 07:36:20] <<< Loading model from trained_models/tutorial_model/epoch_4_Model_Wrapper.pkl ... >>>
[18/12/2021 07:36:20] <<< Loading model from trained_models/tutorial_model/epoch_4.h5 ... >>>
[18/12/2021 07:36:22] <<< Loading optimized model... >>>
[18/12/2021 07:36:28] <<< Optimized model loaded. >>>
[18/12/2021 07:36:28] <<< Model loaded in 7.7805 seconds. >>>


In [28]:
is_transformer = params.get('ATTEND_ON_OUTPUT', 'transformer' in params['MODEL_TYPE'].lower())

params_prediction = {
    'language': 'en',
    'tokenize_f': eval('dataset.' + 'tokenize_none'),
    'beam_size': 12,
    'optimized_search': True,
    'model_inputs': params['INPUTS_IDS_MODEL'],
    'model_outputs': params['OUTPUTS_IDS_MODEL'],
    'dataset_inputs':  params['INPUTS_IDS_DATASET'],
    'dataset_outputs':  params['OUTPUTS_IDS_DATASET'],
    'n_parallel_loaders': 1,
    'maxlen': 50,
    'normalize_probs': True,
    'pos_unk': True and not is_transformer,
    'heuristic': 0,
    'state_below_maxlen': -1,
    'predict_on_sets': ['test'],
    'verbose': 0,
    'attend_on_output': is_transformer
  }
predictions = nmt_model.predictBeamSearchNet(dataset, params_prediction)['test']


[18/12/2021 07:36:57] <<< Predicting outputs of test set >>>




 Total cost: 9421.617930 	 Average cost: 3.144732
The sampling took: 153.640150 secs (Speed: 0.051282 sec/sample)


In [29]:
from keras_wrapper.utils import decode_predictions_beam_search
vocab = dataset.vocabulary['target_text']['idx2words']
samples = predictions['samples'] # Get word indices from the samples.

predictions = decode_predictions_beam_search(samples,  
                                             vocab,
                                             verbose=params['VERBOSE'])

[18/12/2021 07:40:19] Decoding beam search prediction ...


In [30]:
filepath = 'test.pred'
from keras_wrapper.extra.read_write import list2file
list2file(filepath, predictions)
!head -n 4 test.pred

I would like to book a room until tomorrow , please .
please wake us up tomorrow at a quarter past seven .
I am leaving today in the afternoon .
would you mind sending down our luggage to room number three two six , please ?


In [31]:
dataset.setOutput('examples/EuTrans/test.en',
             'test',
             type='text',
             id='target_text',
             pad_on_batch=True,
             tokenization='tokenize_none',
             sample_weights=True,
             max_text_len=30,
             max_words=0)
keep_n_captions(dataset, repeat=1, n=1, set_names=['test'])

[18/12/2021 07:41:39] 	Applying tokenization function: "tokenize_none".
[18/12/2021 07:41:39] Loaded "test" set outputs of data_type "text" with data_id "target_text" and length 2996.
[18/12/2021 07:41:39] Keeping 1 captions per input on the test set.
[18/12/2021 07:41:39] Samples reduced to 2996 in test set.


In [32]:
from keras_wrapper.extra.evaluation import select
metric = 'sacrebleu'
# Apply sampling
extra_vars = dict()
extra_vars['tokenize_f'] = eval('dataset.' + 'tokenize_none')
extra_vars['language'] = params['TRG_LAN']
extra_vars['test'] = dict()
extra_vars['test']['references'] = dataset.extra_variables['test']['target_text']
metrics = select[metric](pred_list=predictions,
                                          verbose=1,
                                          extra_vars=extra_vars,
                                          split='test')

[18/12/2021 07:42:19] Computing SacreBleu scores on the test split...
[18/12/2021 07:42:19] Bleu_4: 65.99690843438069
